In [77]:
# Importar librerías a usar
import wget
import pandas as pd
import dash
import plotly.express as px
from dash import dcc, html
from dash.dependencies import Input, Output

In [78]:
# Descargar, guardar y convertir en dataframe el dataset
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df.drop(columns = ["Unnamed: 0"], inplace = True)
spacex_df.head(10)

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0
5,7,CCAFS LC-40,0,3170.0,F9 v1.1,v1.1
6,8,CCAFS LC-40,0,3325.0,F9 v1.1,v1.1
7,9,CCAFS LC-40,0,2296.0,F9 v1.1,v1.1
8,10,CCAFS LC-40,0,1316.0,F9 v1.1,v1.1
9,11,CCAFS LC-40,0,4535.0,F9 v1.1,v1.1


In [79]:
max_payload = float(spacex_df["Payload Mass (kg)"].max())
min_payload = float(spacex_df["Payload Mass (kg)"].min())

In [80]:
# Inicializar la aplicación de Dash
app = dash.Dash(__name__)

In [81]:
# Definir el diseño de el dashboard
app.layout = html.Div([
                        # Título del dashboard
                        html.H1("SpaceX Launch Records Dashboard",
                                style = {"textAlign": "center"}),

                        # Desplegable para el gráfico circular
                        dcc.Dropdown(id = "site-dropdown",
                                     options = [{"label": "All sites", "value": "ALL"},
                                                {"label": "CCAFS LC-40", "value": "CCAFS LC-40"},
                                                {"label": "VAFB SLC-4E", "value": "VAFB SLC-4E"},
                                                {"label": "KSC LC-39A", "value": "KSC LC-39A"},
                                                {"label": "CCAFS SLC-40", "value": "CCAFS SLC-40"}],
                                     value = "ALL",
                                     placeholder = "All sites",
                                     searchable = True),

                        # Gráfico Circular
                        dcc.Graph(id = "success-pie-chart"),

                        # Deslizador de rango para el gráfico de dispersión
                        dcc.RangeSlider(id = "payload-slider",
                                        min = 0,
                                        max = 10000,
                                        step = 100,
                                        marks = {0:"0", 2500: "2500", 5000: "5000", 7500: "7500", 10000: "10000"},
                                        value = [min_payload, max_payload]),

                        # Gráfico de Dispersión
                        dcc.Graph(id = "success-payload-scatter-chart")
])

In [82]:
# Retrollamada (callback) para actualizar el gráfico circular
@app.callback(Output(component_id = "success-pie-chart", component_property = "figure"),
              Input(component_id = "site-dropdown", component_property = "value"))

def update_pie_chart(selected_site):
  site_data = spacex_df[["Launch Site", "class"]]

  if selected_site == "ALL":
    fig = px.pie(site_data,
                 values = "class",
                 names = "Launch Site",
                 title = "Total Success Launches By Site")
  else:
    site_data = pd.DataFrame(site_data.groupby(["Launch Site"], as_index = False)["class"].value_counts())
    site_data = site_data[site_data["Launch Site"] == selected_site]
    fig = px.pie(site_data,
                 values = "count",
                 names = "class",
                 title = f"Total Success Launches For Site {selected_site}")
  return fig

In [83]:
# Retrollamada (callback) para actualizar el gráfico circular
@app.callback(Output(component_id = "success-payload-scatter-chart", component_property = "figure"),
              [Input(component_id = "site-dropdown", component_property = "value"),
               Input(component_id = "payload-slider", component_property = "value")])

def update_scatter_plot(selected_site, selected_range):

  scatter_data = spacex_df[["Launch Site", "class", "Payload Mass (kg)", "Booster Version Category"]]

  if selected_site == "ALL":
    scatter_data_all = scatter_data[(scatter_data["Payload Mass (kg)"] >= selected_range[0]) &
                                (scatter_data["Payload Mass (kg)"] <= selected_range[1])]
    fig = px.scatter(scatter_data_all,
                     x = "Payload Mass (kg)",
                     y = "class",
                     color = "Booster Version Category",
                     title = "Correlation Between Payload and Success For All Sites")
  else:
    scatter_data_else = scatter_data[(scatter_data["Payload Mass (kg)"] >= selected_range[0]) &
                                     (scatter_data["Payload Mass (kg)"] <= selected_range[1]) &
                                     (scatter_data["Launch Site"] == selected_site)]
    fig = px.scatter(scatter_data_else,
                     x = "Payload Mass (kg)",
                     y = "class",
                     color = "Booster Version Category",
                     title = f"Correlation Between Payload and Success For Site {selected_site}")
  return fig

In [84]:
# Ejectura la aplicación
if __name__ == '__main__':
    app.run_server(debug = True)